In [1]:
%load_ext autoreload
%autoreload 2

import json
import random
import pickle
from datetime import datetime
from heapq import heappush, heappop, heapify, nsmallest, nlargest

import evaluate
import torch
import numpy as np
from datasets import load_dataset
from tqdm import tqdm

from eval import *
from metrics import *
from mistral.generation import Mistral
from mistral.mixed_generation import MixedMistral
from mistral.tokenizer import Tokenizer
from mistral.ngrams.ngram_models import make_models

/gscratch/raivn/ethans/miniconda3/envs/llms_12.1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


In [2]:
# load data
with open("../gpt-2-output-dataset/data/webtext.valid.jsonl", "r") as f:
    dataset = [json.loads(line)["text"] for line in f]
mixing_options = ["sample", "sample_new_weights_with_score", "sample_weights_with_current"]
smoothing_options = [None, "geom", "all"]
data = dataset[:96]

In [3]:
# params
prompt_len = 40
max_gen_len = 10
n_drafts = 3
n_token_sample = n_drafts**2
n_token_consider = 32000
bsz = 16
tokenizer = Tokenizer('./tokenizer.model')
mixing_method = mixing_options[2]
smoothing = smoothing_options[1]
sample_tokens = False
sample_beams = False
diversity_boost = (None, None)

# weighting
ckpt_path = None
ckpt_path = "./ckpts-200k"
if ckpt_path == "./ckpts-100k":
    i_weights = [0.01, 0.12, 0.15, 0.18, 0.06]
    i_length = [1, 2, 3, 4, 5]
    alpha = 0.26 
    temp = 0.68
elif ckpt_path == "./ckpts-200k":
    i_weights = [0.01, 0.04, 0.15, 0.18, 0.12]
    i_length = [1, 2, 3, 4, 5]
    alpha = 0.7
    temp = 0.23
elif ckpt_path == "./ckpts-300k":
    i_weights = [0.01, 0.04, 0.15, 0.18, 0.12]
    i_length = [1, 2, 3, 4, 5]
    alpha = 0.7
    temp = 0.23
else:
    i_weights = None
    i_length = None
    alpha = 0.23
    temp = 0.70

In [4]:
if ckpt_path is not None:
    ngrams = make_models(ckpt_path, bigram=True, trigram=True, fourgram=True, fivegram=True, sixgram=True, sevengram=False)
else:
    ngrams = None

Making bigram...
1310800
Making trigram...
671088728
Making fourgram...
2684354648
Making fivegram...
5368709200
Making sixgram...
5368709200


In [5]:
mixed_device = torch.device("cuda:0")
reg_device = torch.device("cuda:1")

In [6]:
mixed_model = MixedMistral.build(ckpt_dir="./mistral/",
                                 max_batch_size=32,
                                 num_pipeline_ranks=100, 
                                 max_batch_size=32,
                                 device=mixed_device)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


/gscratch/raivn/ethans/miniconda3/envs/llms_12.1/lib/python3.11/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


Loaded in 113.39 seconds


In [12]:
reg_model = Mistral.build(ckpt_dir="./mistral/", 
                          max_batch_size=32,
                          num_pipeline_ranks=100,
                          max_batch_size=32,
                          device=mixed_device)

0
Loaded in 19.87 seconds


In [7]:
alphas = np.arange(0.05, 0.95, 0.05)
temps = np.arange(0.1, 1.4, 0.05)
sample_beam_options = [True, False]
diversity_boost_options = [(None, None)] + [(1, s) for s in np.arange(0.35, 0.95, 0.1)]
mixing_options = ["sample_new_weights_with_score", "sample_weights_with_current"]

# alphas = [0.7]
# temps = [0.23]
# sample_beam_options = [False]
# diversity_boost_options = [(None, None)]
# mixing_options = ["sample_weights_with_current"]
# for alpha in alphas:
#     for temp in temps:
#         for mixing_method in mixing_options:

total_len = len(alphas) * len(temps) * len(mixing_options)
print(total_len)

936


In [8]:
results = {}
loop = tqdm(total=total_len, position=0, leave=True)
for alpha in alphas:
    for temp in temps:
        for mixing_method in mixing_options:
            # params = {"alpha": alpha, "temp": temp, "sample_tokens": sample_tokens, "sample_beam": sample_beams, "diversity_boost": diversity_boost, "mixing_method": mixing_method}
            # output_ppl, alpha, temp, sample_tokens, sample_beams, diversity_boost, mixing_method = param
            mixed_sequences, mixed_ppl = evaluate_mixed_losses(data=data,
                           model=mixed_model,
                           tokenizer=tokenizer,
                           prompt_len=prompt_len,
                           max_gen_len=max_gen_len,
                           alpha=alpha,
                           temp=temp,
                           n_drafts=n_drafts,
                           n_token_consider=n_token_consider,
                           n_token_sample=n_token_sample,
                           mixing_method=mixing_method,
                           smoothing=smoothing,
                           debug=False,
                           bsz=bsz,
                           i_weights=i_weights,
                           i_length=i_length,
                           ngrams=ngrams,
                           sample_beams=sample_beams,
                           diversity_boost=diversity_boost,
                           sample_tokens=sample_tokens)
            param = alpha, temp, sample_tokens, sample_beams, diversity_boost, mixing_method 
            results[param] = mixed_sequences
            loop.set_description(f"alpha: {alpha} temp: {temp}")
            loop.update(1)

alpha: 0.9000000000000001 temp: 1.3500000000000005: 100%|███████████████████████████████| 936/936 [2:49:44<00:00, 11.32s/it]

In [9]:
with open("results/mixed_val_results_7B.pkl", "wb") as f:
    pickle.dump(results, f)

# Checking

In [7]:
torch.set_default_dtype(torch.float32)
mauve = evaluate.load('mauve')

In [8]:
def decode(tokenizer, encoding):
    """
    Args:
        tokenizer (Any): Tokenizer
        encoding (torch.Tensor): Encoding
    Returns:
        decoding (str)
    """
    eos_locs = (encoding == tokenizer.eos_id).nonzero()
    if len(eos_locs > 0):
        encoding = encoding[:eos_locs[0]]
    return tokenizer.decode(encoding.to(torch.int32).tolist())
    
def evaluate_mauve(tokenizer, predictions, references, mauve=True, verbose=True):
    """
    Evaluate mauve.
    Args:
        tokenizer (Any): Tokenizer
        predictions (torch.Tensor): Tokens of predicted sequences, flattened to (n_prompts * n_drafts, gen_len)
        references (List[str]): References
        verbose (bool): Whether to print out examples
    Returns:
        Mauve score
    """
    decoded_preds = []
    prompts = []
    count = 0
    for i in tqdm(range(len(predictions))):
        d = decode(tokenizer, predictions[i])
        decoded_preds.append(d)
        if verbose and i % 1 == 0:
            # first draft of this prompt
            if i % n_drafts == 0:
                count = 0
                print("---------------")
                prompt = decode(tokenizer, predictions[i][:prompt_len])
                print(f"prompt: {prompt}")
            print(f"{count}: {d}")
            count += 1
    if mauve:
        mauve_score = mauve.compute(predictions=decoded_preds, references=references).mauve
        print(mauve_score)

In [14]:
# load from graded
best_params = []
with open("results/mixed_val_results_7B.pkl_graded.pkl", "rb") as f:
    heap = pickle.load(f)
for i in range(20):
    p = heappop(heap)
    print("PPL:" + str(p[0]))
    best_params.append(p[1])

PPL:4.9765625
PPL:5.0703125
PPL:5.234375
PPL:5.26171875
PPL:5.31640625
PPL:5.328125
PPL:5.33203125
PPL:5.36328125
PPL:5.37109375
PPL:5.41796875
PPL:5.42578125
PPL:5.4453125
PPL:5.5234375
PPL:5.52734375
PPL:5.53515625
PPL:5.57421875
PPL:5.58984375
PPL:5.6015625
PPL:5.64453125
PPL:5.6640625


In [16]:
data = dataset[102:112]
for i, param in enumerate(best_params):
    print("----------------------------")
    print(i, param)
    alpha, temp, sample_tokens, sample_beams, diversity_boost, mixing_method = param[0], param[1], param[2], param[3], param[4], param[5]
    mixed_sequences, mixed_ppl = evaluate_mixed_losses(data=data,
                   model=mixed_model,
                   tokenizer=tokenizer,
                   prompt_len=prompt_len,
                   max_gen_len=max_gen_len,
                   alpha=alpha,
                   temp=temp,
                   n_drafts=n_drafts,
                   n_token_consider=n_token_consider,
                   n_token_sample=n_token_sample,
                   mixing_method=mixing_method,
                   smoothing=smoothing,
                   debug=False,
                   bsz=bsz,
                   i_weights=i_weights,
                   i_length=i_length,
                   ngrams=ngrams,
                   sample_beams=sample_beams,
                   diversity_boost=diversity_boost,
                   sample_tokens=sample_tokens)
    mixed_sequences.to(reg_device)
    evaluate_mauve(tokenizer, mixed_sequences.reshape(n_drafts * len(data), -1), None, False)
    # print(calculate_perplexity(reg_model, mixed_sequences.reshape(n_drafts * len(data), -1), prompt_len, bsz=bsz))
    

----------------------------
0 (0.55, 0.1, False, False, (None, None), 'sample_new_weights_with_score')


100%|█████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 3809.66it/s]


---------------
prompt: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaeper
0: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaepernick's protests were "the most important
1: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaepernick's protests were "the most powerful
2: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaepernick's protests were "the most influ
---------------
prompt: TRENTON - Although he has proclaimed his innocence since FBI agents raided his home and City Hall, Trenton Mayor Tony Mack was found guilty today.

A j
0: TRENTON - Although he has proclaimed his innocence since FBI agents raide

100%|█████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 4010.11it/s]


---------------
prompt: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaeper
0: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaepernick's protests were "the most important
1: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaepernick's protests were "the most powerful
2: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaepernick's protests were "a most important
---------------
prompt: TRENTON - Although he has proclaimed his innocence since FBI agents raided his home and City Hall, Trenton Mayor Tony Mack was found guilty today.

A j
0: TRENTON - Although he has proclaimed his innocence since FBI agents rai

...

---------------
prompt: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaeper
0: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaepernick's protests were "the most important
1: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaepernick's protests were "the most powerful
2: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaepernick's protests were "a most important
---------------
prompt: TRENTON - Although he has proclaimed his innocence since FBI agents raided his home and City Hall, Trenton Mayor Tony Mack was found guilty today.

A j
0: TRENTON - Although he has proclaimed his innocence since FBI agents rai

100%|█████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 3923.33it/s]


---------------
prompt: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaeper
0: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaepernick's protests "sparked a national
1: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaepernick's protests "sparked a conversation
2: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaepernick's protests "sparked an national
---------------
prompt: TRENTON - Although he has proclaimed his innocence since FBI agents raided his home and City Hall, Trenton Mayor Tony Mack was found guilty today.

A j
0: TRENTON - Although he has proclaimed his innocence since FBI agents raided his

100%|█████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 4034.80it/s]

---------------
prompt: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaeper
0: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaepernick's protests "sparked a national
1: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaepernick's protests "sparked a conversation
2: GQ magazine named Colin Kaepernick its "citizen of the year" for his launching the last year of NFL kneeling protests. The magazine noted that Kaepernick's protests "sparked an national
---------------
prompt: TRENTON - Although he has proclaimed his innocence since FBI agents raided his home and City Hall, Trenton Mayor Tony Mack was found guilty today.

A j
0: TRENTON - Although he has proclaimed his innocence since FBI agents raided his